In [ ]:
# 📌 Importer les bibliothèques
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 🔹 Lire le fichier CSV
df = pd.read_csv(
    r"C:\Users\hp\Desktop\Datase-fraude\Online_Retail.csv",
    delimiter=";"
)

# 🔹 Nettoyage initial
# Convertir UnitPrice en float
df["UnitPrice"] = df["UnitPrice"].str.replace(",", ".", regex=False).astype(float)

# Convertir InvoiceDate en datetime
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])

# Supprimer les lignes avec quantité <=0 ou prix <=0
df = df[(df['Quantity'] > 0) & (df['UnitPrice'] > 0)]

# Calculer TotalAmount
df['TotalAmount'] = df['Quantity'] * df['UnitPrice']

# Supprimer inf / -inf / NaN
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(subset=['TotalAmount'], inplace=True)

# 🔹 Extraire des informations temporelles
df['Year'] = df['InvoiceDate'].dt.year
df['Month'] = df['InvoiceDate'].dt.month
df['Day'] = df['InvoiceDate'].dt.day
df['Hour'] = df['InvoiceDate'].dt.hour
df['DayOfWeek'] = df['InvoiceDate'].dt.dayofweek
df['MonthYear'] = df['InvoiceDate'].dt.to_period('M')

# 🔹 Aperçu des données
print("Valeurs manquantes :\n", df.isnull().sum())
print("\nDescription statistique :\n", df.describe())
print("\nPremières lignes :\n", df.head())

# 🔹 Analyse par pays
country_counts = df['Country'].value_counts()
plt.figure(figsize=(10,5))
sns.barplot(x=country_counts.index[:10], y=country_counts.values[:10])
plt.xticks(rotation=45)
plt.title("Top 10 pays par nombre de transactions")
plt.show()

# 🔹 Histogramme des montants des transactions
plt.figure(figsize=(10,5))
sns.histplot(df['TotalAmount'], bins=100, log_scale=(True, True))
plt.title("Distribution des montants des transactions")
plt.xlabel("Montant Total")
plt.ylabel("Nombre de transactions")
plt.show()

# 🔹 Évolution des ventes par mois
monthly_sales = df.groupby('MonthYear')['TotalAmount'].sum()
monthly_sales.plot(figsize=(12,5), marker='o')
plt.title("Évolution des ventes par mois")
plt.xlabel("Mois")
plt.ylabel("Montant total des ventes")
plt.show()

# 🔹 Retours / factures commençant par 'C'
returns = df[df['InvoiceNo'].str.startswith('C')]
print(f"Nombre de retours : {returns.shape[0]}")

# 🔹 Top produits par quantité vendue
top_products = df.groupby('Description')['Quantity'].sum().sort_values(ascending=False).head(10)
plt.figure(figsize=(10,5))
sns.barplot(x=top_products.values, y=top_products.index)
plt.title("Top 10 produits vendus")
plt.xlabel("Quantité vendue")
plt.show()
